# Reto 01

> Consiste en encontrar la carpeta de descargas del usuario (independientemente de sus personalizaciones, tiene que funcionar si la tiene en otro lado o es un usuario cualquiera) y listar solo los archivos de la carpeta.


Sacado de https://j2logo.com/python/listar-directorio-en-python/

Para listar o recorrer un directorio:

In [2]:
import os
contenido = os.listdir('/Users/noave/Downloads')
contenido

['backupsTabGroupsFirefox',
 'borrar-cuando-termine-reto.txt',
 'desktop.ini',
 'oso-trospido.jpg',
 'Packs tipografías descargadas',
 'Telegram Desktop',
 '_editores texto y código',
 '_Libros',
 '_Portátil',
 '_Servicios Editoriales',
 '_Varios']

El primer problema es que me muestra también todas las carpetas. El segundo, que aún no sé cómo encontrar un directorio con un comodín. Al parecer, en Windows requiere acceder al registro, pero he leído que hay otro método. 

Para mostrar los archivos de tipo jpg de una carpeta:

In [3]:
import os
ejemplo_dir = '/Users/noave/Downloads'
contenido = os.listdir(ejemplo_dir)
imagenes = []
for fichero in contenido:
    if os.path.isfile(os.path.join(ejemplo_dir, fichero)) and fichero.endswith('.jpg'):
        imagenes.append(fichero)
print(imagenes)

['oso-trospido.jpg']


Con scandir() puedo listar los ficheros de un directorio. Es la opción recomendada.

In [4]:
import os
ejemplo_dir = '/Users/noave/Downloads'
with os.scandir(ejemplo_dir) as ficheros:
    for fichero in ficheros:
        print(fichero.name)

backupsTabGroupsFirefox
borrar-cuando-termine-reto.txt
desktop.ini
oso-trospido.jpg
Telegram Desktop
_editores texto y código
_Libros
_Portátil
_Servicios Editoriales
_Varios


Y también está iterdir() del módulo pathlib, también recomendado y que permite filtrar con is_file() o is_dir().

In [5]:
import pathlib
ejemplo_dir = '/Users/noave/Downloads'
directorio = pathlib.Path(ejemplo_dir)
for fichero in directorio.iterdir():
    print(fichero.name)

backupsTabGroupsFirefox
borrar-cuando-termine-reto.txt
desktop.ini
oso-trospido.jpg
Telegram Desktop
_editores texto y código
_Libros
_Portátil
_Servicios Editoriales
_Varios


Para recorrer solo los ficheros de un directorio, dos métodos con lo que hemos visto. 

In [7]:
import os
ejemplo_dir = '/Users/noave/Downloads'
with os.scandir(ejemplo_dir) as ficheros:
    ficheros = [fichero.name for fichero in ficheros if fichero.is_file()]
print(ficheros)

['borrar-cuando-termine-reto.txt', 'desktop.ini', 'oso-trospido.jpg']


In [14]:
import pathlib

ejemplo_dir = '/Users/noave/Downloads/'

directorio = pathlib.Path(ejemplo_dir)
ficheros = [fichero.name for fichero in directorio.iterdir() if fichero.is_file()]
for i in ficheros:
    print(i)

borrar-cuando-termine-reto.txt
desktop.ini
oso-trospido.jpg


In [10]:
#Y si solo queremos las carpetas:

import os
ejemplo_dir = '/Users/noave/Downloads/'
with os.scandir(ejemplo_dir) as ficheros:
    subdirectorios = [fichero.name for fichero in ficheros if fichero.is_dir()]
print(subdirectorios)

['backupsTabGroupsFirefox', 'Telegram Desktop', '_editores texto y código', '_Libros', '_Portátil', '_Servicios Editoriales', '_Varios']


Pues ya tenemos un ganador. Ahora solo nos falta descubrir la manera de que Python encuentre la carpeta de descargas, esté donde esté. 

In [18]:
DIRECTORIO_BASE_2 = os.getcwd()
print(DIRECTORIO_BASE_2)


C:\Users\noave\Python


https://stackoverflow.com/questions/35851281/python-finding-the-users-downloads-folder

Alguien dio esta posible solución, pero como toca registro, no me atrevo a meterla tal cual:

```import os

def get_download_path():
    """Returns the default downloads path for linux or windows"""
    if os.name == 'nt':
        import winreg
        sub_key = r'SOFTWARE\Microsoft\Windows\CurrentVersion\Explorer\Shell Folders'
        downloads_guid = '{374DE290-123F-4565-9164-39C4925E467B}'
        with winreg.OpenKey(winreg.HKEY_CURRENT_USER, sub_key) as key:
            location = winreg.QueryValueEx(key, downloads_guid)[0]
        return location
    else:
        return os.path.join(os.path.expanduser('~'), 'downloads')
```

Este me inspira más confianza, aunque de novato no tiene nada:

```
import os

if os.name == 'nt':
    import ctypes
    from ctypes import windll, wintypes
    from uuid import UUID

    #ctypes GUID copied from MSDN sample code
    class GUID(ctypes.Structure):
        _fields_ = [
            ("Data1", wintypes.DWORD),
            ("Data2", wintypes.WORD),
            ("Data3", wintypes.WORD),
            ("Data4", wintypes.BYTE * 8)
        ] 

        def __init__(self, uuidstr):
            uuid = UUID(uuidstr)
            ctypes.Structure.__init__(self)
            self.Data1, self.Data2, self.Data3, \
                self.Data4[0], self.Data4[1], rest = uuid.fields
            for i in range(2, 8):
                self.Data4[i] = rest>>(8-i-1)*8 & 0xff

    SHGetKnownFolderPath = windll.shell32.SHGetKnownFolderPath
    SHGetKnownFolderPath.argtypes = [
        ctypes.POINTER(GUID), wintypes.DWORD,
        wintypes.HANDLE, ctypes.POINTER(ctypes.c_wchar_p)
    ]

    def _get_known_folder_path(uuidstr):
        pathptr = ctypes.c_wchar_p()
        guid = GUID(uuidstr)
        if SHGetKnownFolderPath(ctypes.byref(guid), 0, 0, ctypes.byref(pathptr)):
            raise ctypes.WinError()
        return pathptr.value

    FOLDERID_Download = '{374DE290-123F-4565-9164-39C4925E467B}'

    def get_download_folder():
        return _get_known_folder_path(FOLDERID_Download)
else:
    def get_download_folder():
        home = os.path.expanduser("~")
        return os.path.join(home, "Downloads")
        
print(get_download_folder())
```
        

In [23]:
def get_download_folder():
        home = os.path.expanduser("~")
        return os.path.join(home, "Downloads")
        
print(get_download_folder())

C:\Users\noave\Downloads


Vale, me ha funcionado. Supongo que si moviese la carpeta a otra parte, dejaría de funcionar (en realidad es la ruta por defecto home+downloads). Pero ya es algo. 

Ahora solo me falta… Hacer que imprima los archivos de ese directorio. 

In [25]:
import pathlib

def get_download_folder():
        home = os.path.expanduser("~")
        return os.path.join(home, "Downloads")
    
print(get_download_folder())        

ejemplo_dir = get_download_folder()
directorio = pathlib.Path(ejemplo_dir)
ficheros = [fichero.name for fichero in directorio.iterdir() if fichero.is_file()]
for i in ficheros:
    print(i)

C:\Users\noave\Downloads
borrar-cuando-termine-reto.txt
desktop.ini
oso-trospido.jpg


Reto conseguido. ¿Funcionará con otros usuarios?

In [55]:
import os

def get_download_folder():
        home = os.path.expanduser("~")
        return os.path.join(home, "Downloads")
    
print(get_download_folder()) 

ejemplo_dir = get_download_folder()
with os.scandir(ejemplo_dir) as ficheros:
    ficheros = [fichero.name for fichero in ficheros if fichero.is_file()]
for i in ficheros:
    print(i)

C:\Users\noave\Downloads
borrar-cuando-termine-reto.txt
desktop.ini
oso-trospido.jpg


---
# Reto 2

> En este caso, y partiendo de la solución anterior, ahora hay que trabajar con archivos. El objetivo es que listes todos los archivos JPEG que se encuentran en el directorio de descargas del usuario.

> Tienes que tener en cuenta, que imágenes JPEG, son los .jpg o .jpeg o incluso JPEG por citarte algunas. Pero además en el listado se cumplirán las siguientes condiciones,
> - Si en el nombre del archivo aparece un número, el nombre del achivo apararecerá en mayúsculas, mientras que si no tiene ningún número aparecerá en minúsculas.
> - Además en ese listado, los que sean pares, tendrán una flecha delante, del tipo =>.

> Como ves se trata de un reto sencillo, y que nos permitirá profundizar en bucles, módulos y otros temas similares.

Este reto tiene una parte que ya está solucionada más arriba, en mis notas del reto 1. Es la parte de los archivos en JPG. Para extender las variaciones dentro de la extensión (jpeg, JPEG, jpg, etc.), tal vez pueda aplicar expresiones regulares. No sé si algo como jp*g funcionará, imagino que tendría que importar el módulo y ver cuál es la forma correcta. Y, de paso, tal vez tenga una forma de evitar el case sensitive sin recurrir a .lower().

También habrá que hacer un bucle for para cada carácter dentro de cada archivo, un for i in str, para determinar si hay números. Aquí imagino que sí necesitaré expresiones regulares, ya que al ser str no puedo preguntar si hay int. Necesitaría filtrar con [0-9]. Si la respuesta es True: upper(str). 

Por último, hay varias formas de hacer algo con los elementos pares o impares. Se puede hacer con bucles while tanto con for. Por lo general, se determinan los pares cuando su index % 2 = 0. O bien podemos establecer un range(start, end, 2), pero no me acuerdo de si puedo usar el valor len(archivos) para determinar el end del range. 

Empecemos.

Antes de andar jugueteando con mis directorios, voy a simular con una lista.


In [48]:
# Encontrar archivos cuya extensión coincida con .jpg y variantes:
import re
archivos = ['imagen01.jpg', 'oso_trospido.JPG', "Karaoke.mp3", 'Ashjkdaslkl.JPEG', 'AKAKAKAKA9.jpeg', 'final_image.jpg']
patron = re.compile(r"\.jp[e]?g", re.IGNORECASE)

prueba = "prueba.JPEG"
print(patron.search(prueba)) # No sé por qué, pero .match() me da siempre None.
print(prueba, "\n---------------")

for i in archivos:
    print(patron.search(i))
    if patron.search(i):
        print(i)



<re.Match object; span=(6, 11), match='.JPEG'>
prueba.JPEG 
---------------
<re.Match object; span=(8, 12), match='.jpg'>
imagen01.jpg
<re.Match object; span=(12, 16), match='.JPG'>
oso_trospido.JPG
None
<re.Match object; span=(11, 16), match='.JPEG'>
Ashjkdaslkl.JPEG
<re.Match object; span=(10, 15), match='.jpeg'>
AKAKAKAKA9.jpeg
<re.Match object; span=(11, 15), match='.jpg'>
final_image.jpg


In [49]:
# Determinar si los archivos llevan números.
# Si es True, ponerlo todo en mayúsculas.
import re
archivos = ['imagen01.jpg', 'oso_trospido.JPG',
            'Ashjkdaslkl.JPEG', 'AKAKAKAKA9.jpeg', 'final_image.jpg']
patron = re.compile(r"[0-9]", re.IGNORECASE)

prueba = "imagen5.jpg"
print(patron.search(prueba))
print(prueba, "\n---------------")

for i in archivos:
#    print(patron.search(i))
    if patron.search(i):
        print(i.upper())
    else:
        print(i)

<re.Match object; span=(6, 7), match='5'>
imagen5.jpg 
---------------
IMAGEN01.JPG
oso_trospido.JPG
Ashjkdaslkl.JPEG
AKAKAKAKA9.JPEG
final_image.jpg


In [54]:
# Si es par, añadir una flecha.

archivos = ['imagen01.jpg', 'oso_trospido.JPG',
            'Ashjkdaslkl.JPEG', 'AKAKAKAKA9.jpeg', 'final_image.jpg']

pos=0
for i in archivos:
    pos+=1
    if pos%2==0:
        print("=> ", i)
    else:
        print(i)        

imagen01.jpg
=>  oso_trospido.JPG
Ashjkdaslkl.JPEG
=>  AKAKAKAKA9.jpeg
final_image.jpg


Ya tendríamos prácticamente todo por separado. Vamos a ponerlo todo junto. 

In [66]:
import os
import re

def get_download_folder():
    home = os.path.expanduser("~")
    return os.path.join(home, "Downloads")

print(get_download_folder())

patronJPG = re.compile(r"\.jp[e]?g", re.IGNORECASE)
patronNum = re.compile(r"[0-9]", re.IGNORECASE)
descargasDir = get_download_folder()

contenido = os.listdir(descargasDir)
imagenes = []
for fichero in contenido:
    if os.path.isfile(os.path.join(descargasDir, fichero)) and patronJPG.search(fichero):
        imagenes.append(fichero)

pos = 0

for i in imagenes:
    pos += 1
    if pos % 2 == 0:
        i = "=> " + i

    if patronNum.search(i):
        print(i.upper())
    else:
        print(i.lower())

C:\Users\noave\Downloads
2021-06-10 AT 20.24.57.JPEG
=> 808187.JPG
images.jpg
=> oso-trospido.jpg
PEXELS-C-TECHNICAL-6636897.JPG
=> punch-a-nazi.jpg
sogadesedaymagia_tradingcard-wombo.jpg
=> takashi-akira.jpg
WHATSAPP IMAGE 2021-04-30 AT 21.14.49.JPEG


¡Reto número 2 conseguido!

---

`[Actualización 220307]`


Al hacer Pull Request, Github me da el siguiente mensaje a través de [pep8speaks](https://github.com/pep8speaks):

> Hello @NoaVelasco! Thanks for opening this PR. We checked the lines you've touched for PEP 8 issues, and found:

    - In the file reto-01/NoaVelasco/reto01.py:

    Line 3:1: E302 expected 2 blank lines, found 1
    Line 4:9: E117 over-indented
    Line 6:1: W293 blank line contains whitespace
    Line 7:1: E305 expected 2 blank lines after class or function definition, found 1
    Line 7:29: W291 trailing whitespace
    Line 13:13: W292 no newline at end of file

    - In the file reto-02/NoaVelasco/reto02.py:

    Line 19:80: E501 line too long (89 > 79 characters)

Así que he hecho algunos cambios para dejar el código lo mejor posible.